# Adding  More negative patients

Adding more negative patients from Mark
creat new `Dataset11` to train

In [1]:
import os
print(os.getcwd())
import imgaug as ia
from imgaug import augmenters as iaa
import skimage.io
from shutil import copy
import errno
import numpy as np
import skimage.color as color
import matplotlib.pyplot as plt
import numpy as np
import os
from skimage.color import gray2rgb

/Users/mingrenshen/Projects/MedicalImgAnalysis/notebooks


In [2]:
# Temporary changing working directory
os.chdir('..')

In [9]:
# Adding more new patients in training
# Label String
class_0_string = "nonbleeding"
class_1_string = "bleeding"

# Processed Results Directory
# root directory
raw_root_dir = "./data/DataSet11/raw_patient_based/"
imgId = 391 # We name patient images from following old training data set
total_num_non_bleeding = 0
destDir = "./data/DataSet11/newNegPatients"

# Process NonBleeding Cases
with open ( "log_new_NonBleeding.csv" , 'w' ) as non_csv_file :
    for patient in os.listdir(raw_root_dir + "DSASeries" ):
        if patient != '.DS_Store':
            dataDir = raw_root_dir  + "DSASeries" + '/' + patient
            for f in os.listdir ( dataDir ):
                if f != '.DS_Store':
                    #print(f)
                    img_new_name = class_0_string + '.' + str(imgId) + ".jpg"
                    #print(img_new_name)
                    copy(dataDir+'/'+f, destDir)
                    os.rename(destDir+'/'+f, destDir+'/'+img_new_name)
                    non_csv_file.write("%s, %s, %s, %d \n" % (img_new_name, patient, f, 0))
                    imgId += 1
                    total_num_non_bleeding += 1
print(total_num_non_bleeding)

/Users/mingrenshen/Projects/MedicalImgAnalysis
/Users/mingrenshen/Projects/MedicalImgAnalysis
/Users/mingrenshen/Projects/MedicalImgAnalysis
/Users/mingrenshen/Projects/MedicalImgAnalysis
/Users/mingrenshen/Projects/MedicalImgAnalysis
/Users/mingrenshen/Projects/MedicalImgAnalysis
/Users/mingrenshen/Projects/MedicalImgAnalysis
/Users/mingrenshen/Projects/MedicalImgAnalysis
/Users/mingrenshen/Projects/MedicalImgAnalysis
/Users/mingrenshen/Projects/MedicalImgAnalysis
/Users/mingrenshen/Projects/MedicalImgAnalysis
/Users/mingrenshen/Projects/MedicalImgAnalysis
/Users/mingrenshen/Projects/MedicalImgAnalysis
/Users/mingrenshen/Projects/MedicalImgAnalysis
/Users/mingrenshen/Projects/MedicalImgAnalysis
/Users/mingrenshen/Projects/MedicalImgAnalysis
/Users/mingrenshen/Projects/MedicalImgAnalysis
/Users/mingrenshen/Projects/MedicalImgAnalysis
/Users/mingrenshen/Projects/MedicalImgAnalysis
/Users/mingrenshen/Projects/MedicalImgAnalysis
/Users/mingrenshen/Projects/MedicalImgAnalysis
/Users/mingre

In [13]:
print(imgId)

610


## Since we notice the file name bugs in previous generating codes, here we re-generating the data augmentation results of the original files

In total we have three files that needs to be regenerated

* old train bleeding
* old train non bleeding
* new added train non bleeding

In [3]:
def getImageList(imageTXT):
    """
    Function to loop the testing images for test
    :param imageTXT: the txt that stores the 
    :return: imageFileList: the list contains all the original test image list
    """
    imageFileList = list()
    with open(imageTXT,'r') as f:
        lines = f.readlines()
        for line in lines:
            imageFileList.append(line.strip())
    return imageFileList

In [4]:
def createFolder(folderName):
    """
    Safely create folder when needed
    
    :param folderName : the directory that you  want to safely create
    :return: None
    """
    if not os.path.exists(folderName):
        try:
            os.makedirs(folderName)
        except OSError as exc:  # Guard against race condition
            if exc.errno != errno.EEXIST:
                raise

### 1. old training bleedin

In [6]:
# 1001's images are in dataPath
dataPath = "./data/DataSet11/fastAITrain/train/Bleeding_0/"
imgTXT = 'bleeding.txt'
current_class = 'bleeding'
startID = 773

##################################################
# 1. Gaussion Noise
##################################################

## Processing  
trainImageTxtFile = dataPath + imgTXT
imageList = getImageList(trainImageTxtFile)
current_operation = "GaussianNoise"

# Add gaussian noise.
# For 50% of all images, we sample the noise once per pixel.
# For the other 50% of all images, we sample the noise per pixel AND
# channel. This can change the color (not only brightness) of the
# pixels.
   
from imgaug import augmenters as iaa
ia.seed(1)
seq = iaa.Sequential([
        iaa.AdditiveGaussianNoise(loc=0, 
                                  scale=(0.0, 0.05*255), 
                                  per_channel=0.5)
])
# Make our sequence deterministic.
# We can now apply it to the image and then to the BBs and it will
# lead to the same augmentations.
# IMPORTANT: Call this once PER BATCH, otherwise you will always get the exactly same augmentations for every batch!
seq_det = seq.to_deterministic()

for img in imageList:
    # print(img)
    # Grayscale images must have shape (height, width, 1) each.
    #print(os.listdir(dataPath+'images/'))
    currentimage = skimage.io.imread(dataPath + img).astype(np.uint8)
    # gray2rgb() simply duplicates the gray values over the three color channels.
    currentimage = color.gray2rgb(currentimage)
    # bbs = bboxSetupInImage(dataPath , img.rstrip('.jpg') + '.txt',currentimage)
    # Augment BBs and images.
    # As we only have one image and list of BBs, we use
    # [image] and [bbs] to turn both into lists (batches) for the# functions and then [0] to reverse that. In a real experiment, your
    # variables would likely already be lists.
    image_aug = seq_det.augment_images([currentimage])[0]
    # bbs_aug = seq_det.augment_bounding_boxes([bbs])[0]
    augImgFolder = current_operation + "Images_" + current_class
    # augTxTFolder = current_operation + "TXT"
    createFolder(augImgFolder)
    # createFolder(augTxTFolder)
    # Save aug images and bboxes
    skimage.io.imsave(augImgFolder + '/' + current_class + '.'  + str(startID) + '.jpg',image_aug)
    startID += 1
print('Done ' + current_operation)

##################################################
# 2. Gaussion Blur
##################################################

current_operation = "GaussianBlur"

# blur images with a sigma of 0 to 3.0  
from imgaug import augmenters as iaa
ia.seed(1)
seq = iaa.Sequential([
  iaa.GaussianBlur(sigma=(0, 3)) 
])

# Make our sequence deterministic.
# We can now apply it to the image and then to the BBs and it will
# lead to the same augmentations.
# IMPORTANT: Call this once PER BATCH, otherwise you will always get the exactly same augmentations for every batch!
seq_det = seq.to_deterministic()

for img in imageList:
    #print(img)
    # Grayscale images must have shape (height, width, 1) each.
    currentimage = skimage.io.imread(dataPath + img).astype(np.uint8)
    # gray2rgb() simply duplicates the gray values over the three color channels.
    currentimage = color.gray2rgb(currentimage)
    image_aug = seq_det.augment_images([currentimage])[0]
    augImgFolder = current_operation + "Images_" + current_class
    createFolder(augImgFolder)
    # Save aug images
    skimage.io.imsave(augImgFolder + '/' + current_class + '.' + str(startID) + '.jpg',image_aug)
    startID += 1
print('Done ' + current_operation)

##################################################
# 3. Brightness
##################################################

current_operation = "Brightness"

# Strengthen or weaken the contrast in each image.   
from imgaug import augmenters as iaa
ia.seed(1)
seq = iaa.Sequential([
    iaa.Multiply((1.2, 1.5))
])

# seq = iaa.Sequential([ 
#     # Adjust contrast by scaling each pixel value to (I_ij/255.0)**gamma.
#     # Values in the range gamma=(0.5, 2.0) seem to be sensible.
#     iaa.GammaContrast((0.5, 1.5))
# ])

# Make our sequence deterministic.
# We can now apply it to the image and then to the BBs and it will
# lead to the same augmentations.
# IMPORTANT: Call this once PER BATCH, otherwise you will always get the exactly same augmentations for every batch!
seq_det = seq.to_deterministic()

for img in imageList:
    #print(img)
    # Grayscale images must have shape (height, width, 1) each.
    currentimage = skimage.io.imread(dataPath+ img).astype(np.uint8)
    # gray2rgb() simply duplicates the gray values over the three color channels.
    currentimage = color.gray2rgb(currentimage)
    image_aug = seq_det.augment_images([currentimage])[0]
    augImgFolder = current_operation + "Images_" + current_class
    createFolder(augImgFolder)
    # Save aug images
    skimage.io.imsave(augImgFolder + '/' + current_class + '.' + str(startID) + '.jpg',image_aug)
    startID += 1
print('Done ' + current_operation)

##################################################
# 4. ContrastNormalization
##################################################

current_operation = "ContrastNormalization"

# Improve or worsen the contrast of images.

from imgaug import augmenters as iaa
ia.seed(1)
seq = iaa.Sequential([
    iaa.ContrastNormalization((0.8, 1.5), per_channel=True)
])

# seq = iaa.Sequential([ 
#     # Adjust contrast by scaling each pixel value to (I_ij/255.0)**gamma.
#     # Values in the range gamma=(0.5, 2.0) seem to be sensible.
#     iaa.GammaContrast((0.5, 1.5))
# ])

# Make our sequence deterministic.
# We can now apply it to the image and then to the BBs and it will
# lead to the same augmentations.
# IMPORTANT: Call this once PER BATCH, otherwise you will always get the exactly same augmentations for every batch!
seq_det = seq.to_deterministic()

for img in imageList:
    #print(img)
    # Grayscale images must have shape (height, width, 1) each.
    currentimage = skimage.io.imread(dataPath +img).astype(np.uint8)
    # gray2rgb() simply duplicates the gray values over the three color channels.
    currentimage = color.gray2rgb(currentimage)
    image_aug = seq_det.augment_images([currentimage])[0]
    augImgFolder = current_operation + "Images_" + current_class
    createFolder(augImgFolder)
    # Save aug images
    skimage.io.imsave(augImgFolder + '/' + current_class + '.' + str(startID) + '.jpg',image_aug)
    startID += 1
print('Done ' + current_operation)

##################################################
# 5. Fliplr
##################################################

current_operation = "Fliplr"

# Flip/mirror input images horizontally.
   
from imgaug import augmenters as iaa
ia.seed(1)
seq = iaa.Sequential([
        iaa.Fliplr(1.0)
])

# seq = iaa.Sequential([ 
#     # Adjust contrast by scaling each pixel value to (I_ij/255.0)**gamma.
#     # Values in the range gamma=(0.5, 2.0) seem to be sensible.
#     iaa.GammaContrast((0.5, 1.5))
# ])

# Make our sequence deterministic.
# We can now apply it to the image and then to the BBs and it will
# lead to the same augmentations.
# IMPORTANT: Call this once PER BATCH, otherwise you will always get the exactly same augmentations for every batch!
seq_det = seq.to_deterministic()

for img in imageList:
    #print(img)
    # Grayscale images must have shape (height, width, 1) each.
    currentimage = skimage.io.imread(dataPath+img).astype(np.uint8)
    # gray2rgb() simply duplicates the gray values over the three color channels.
    currentimage = color.gray2rgb(currentimage)
    image_aug = seq_det.augment_images([currentimage])[0]
    augImgFolder = current_operation + "Images_" + current_class
    createFolder(augImgFolder)
    # Save aug images
    skimage.io.imsave(augImgFolder + '/' + current_class + '.' + str(startID) + '.jpg',image_aug)
    startID += 1
print('Done ' + current_operation)

##################################################
# 6. Flipud
##################################################

current_operation = "Flipud"

# Flip/mirror input images vertically.
   
from imgaug import augmenters as iaa
ia.seed(1)
seq = iaa.Sequential([
        iaa.Flipud(1.0)
])

# seq = iaa.Sequential([ 
#     # Adjust contrast by scaling each pixel value to (I_ij/255.0)**gamma.
#     # Values in the range gamma=(0.5, 2.0) seem to be sensible.
#     iaa.GammaContrast((0.5, 1.5))
# ])

# Make our sequence deterministic.
# We can now apply it to the image and then to the BBs and it will
# lead to the same augmentations.
# IMPORTANT: Call this once PER BATCH, otherwise you will always get the exactly same augmentations for every batch!
seq_det = seq.to_deterministic()

for img in imageList:
    #print(img)
    # Grayscale images must have shape (height, width, 1) each.
    currentimage = skimage.io.imread(dataPath+img).astype(np.uint8)
    # gray2rgb() simply duplicates the gray values over the three color channels.
    currentimage = color.gray2rgb(currentimage)
    image_aug = seq_det.augment_images([currentimage])[0]
    augImgFolder = current_operation + "Images_" + current_class
    createFolder(augImgFolder)
    # Save aug images
    skimage.io.imsave(augImgFolder + '/' + current_class + '.' + str(startID) + '.jpg',image_aug)
    startID += 1
print('Done ' + current_operation)

##################################################
# 7. Rot90or270Degree
##################################################

current_operation = "Rot90or270Degree"

# Rotates all images by 90 or 270 degrees. 
   
from imgaug import augmenters as iaa
ia.seed(1)
seq = iaa.Sequential([
    iaa.Rot90([1, 3])
])

# seq = iaa.Sequential([ 
#     # Adjust contrast by scaling each pixel value to (I_ij/255.0)**gamma.
#     # Values in the range gamma=(0.5, 2.0) seem to be sensible.
#     iaa.GammaContrast((0.5, 1.5))
# ])

# Make our sequence deterministic.
# We can now apply it to the image and then to the BBs and it will
# lead to the same augmentations.
# IMPORTANT: Call this once PER BATCH, otherwise you will always get the exactly same augmentations for every batch!
seq_det = seq.to_deterministic()

for img in imageList:
    #print(img)
    # Grayscale images must have shape (height, width, 1) each.
    currentimage = skimage.io.imread(dataPath+img).astype(np.uint8)
    # gray2rgb() simply duplicates the gray values over the three color channels.
    currentimage = color.gray2rgb(currentimage)
    image_aug = seq_det.augment_images([currentimage])[0]
    augImgFolder = current_operation + "Images_" + current_class
    createFolder(augImgFolder)
    # Save aug images
    skimage.io.imsave(augImgFolder + '/' + current_class + '.'  + str(startID) + '.jpg',image_aug)
    startID += 1
print('Done ' + current_operation)

Done GaussianNoise
Done GaussianBlur
Done 
Done ContrastNormalization
Done Fliplr
Done Flipud
Done Rot90or270Degree


In [7]:
# Print Out Number of bleeding images after data Augmentation
print(startID)

6177


### Old Non Bleeding Patients

In [8]:
# 1001's images are in dataPath
dataPath = "./data/DataSet11/fastAITrain/train/nonBleeding_0/"
imgTXT = 'nonbleeding.txt'
current_class = 'nonbleeding'
startID = 391

##################################################
# 1. Gaussion Noise
##################################################

## Processing  
trainImageTxtFile = dataPath + imgTXT
imageList = getImageList(trainImageTxtFile)
current_operation = "GaussianNoise"

# Add gaussian noise.
# For 50% of all images, we sample the noise once per pixel.
# For the other 50% of all images, we sample the noise per pixel AND
# channel. This can change the color (not only brightness) of the
# pixels.
   
from imgaug import augmenters as iaa
ia.seed(1)
seq = iaa.Sequential([
        iaa.AdditiveGaussianNoise(loc=0, 
                                  scale=(0.0, 0.05*255), 
                                  per_channel=0.5)
])
# Make our sequence deterministic.
# We can now apply it to the image and then to the BBs and it will
# lead to the same augmentations.
# IMPORTANT: Call this once PER BATCH, otherwise you will always get the exactly same augmentations for every batch!
seq_det = seq.to_deterministic()

for img in imageList:
    # print(img)
    # Grayscale images must have shape (height, width, 1) each.
    #print(os.listdir(dataPath+'images/'))
    currentimage = skimage.io.imread(dataPath + img).astype(np.uint8)
    # gray2rgb() simply duplicates the gray values over the three color channels.
    currentimage = color.gray2rgb(currentimage)
    # bbs = bboxSetupInImage(dataPath , img.rstrip('.jpg') + '.txt',currentimage)
    # Augment BBs and images.
    # As we only have one image and list of BBs, we use
    # [image] and [bbs] to turn both into lists (batches) for the# functions and then [0] to reverse that. In a real experiment, your
    # variables would likely already be lists.
    image_aug = seq_det.augment_images([currentimage])[0]
    # bbs_aug = seq_det.augment_bounding_boxes([bbs])[0]
    augImgFolder = current_operation + "Images_" + current_class
    # augTxTFolder = current_operation + "TXT"
    createFolder(augImgFolder)
    # createFolder(augTxTFolder)
    # Save aug images and bboxes
    skimage.io.imsave(augImgFolder + '/' + current_class + '.'  + str(startID) + '.jpg',image_aug)
    startID += 1
print('Done ' + current_operation)

##################################################
# 2. Gaussion Blur
##################################################

current_operation = "GaussianBlur"

# blur images with a sigma of 0 to 3.0  
from imgaug import augmenters as iaa
ia.seed(1)
seq = iaa.Sequential([
  iaa.GaussianBlur(sigma=(0, 3)) 
])

# Make our sequence deterministic.
# We can now apply it to the image and then to the BBs and it will
# lead to the same augmentations.
# IMPORTANT: Call this once PER BATCH, otherwise you will always get the exactly same augmentations for every batch!
seq_det = seq.to_deterministic()

for img in imageList:
    #print(img)
    # Grayscale images must have shape (height, width, 1) each.
    currentimage = skimage.io.imread(dataPath + img).astype(np.uint8)
    # gray2rgb() simply duplicates the gray values over the three color channels.
    currentimage = color.gray2rgb(currentimage)
    image_aug = seq_det.augment_images([currentimage])[0]
    augImgFolder = current_operation + "Images_" + current_class
    createFolder(augImgFolder)
    # Save aug images
    skimage.io.imsave(augImgFolder + '/' + current_class + '.' + str(startID) + '.jpg',image_aug)
    startID += 1
print('Done ' + current_operation)

##################################################
# 3. Brightness
##################################################

current_operation = "Brightness"

# Strengthen or weaken the contrast in each image.   
from imgaug import augmenters as iaa
ia.seed(1)
seq = iaa.Sequential([
    iaa.Multiply((1.2, 1.5))
])

# seq = iaa.Sequential([ 
#     # Adjust contrast by scaling each pixel value to (I_ij/255.0)**gamma.
#     # Values in the range gamma=(0.5, 2.0) seem to be sensible.
#     iaa.GammaContrast((0.5, 1.5))
# ])

# Make our sequence deterministic.
# We can now apply it to the image and then to the BBs and it will
# lead to the same augmentations.
# IMPORTANT: Call this once PER BATCH, otherwise you will always get the exactly same augmentations for every batch!
seq_det = seq.to_deterministic()

for img in imageList:
    #print(img)
    # Grayscale images must have shape (height, width, 1) each.
    currentimage = skimage.io.imread(dataPath+ img).astype(np.uint8)
    # gray2rgb() simply duplicates the gray values over the three color channels.
    currentimage = color.gray2rgb(currentimage)
    image_aug = seq_det.augment_images([currentimage])[0]
    augImgFolder = current_operation + "Images_" + current_class
    createFolder(augImgFolder)
    # Save aug images
    skimage.io.imsave(augImgFolder + '/' + current_class + '.' + str(startID) + '.jpg',image_aug)
    startID += 1
print('Done ' + current_operation)

##################################################
# 4. ContrastNormalization
##################################################

current_operation = "ContrastNormalization"

# Improve or worsen the contrast of images.

from imgaug import augmenters as iaa
ia.seed(1)
seq = iaa.Sequential([
    iaa.ContrastNormalization((0.8, 1.5), per_channel=True)
])

# seq = iaa.Sequential([ 
#     # Adjust contrast by scaling each pixel value to (I_ij/255.0)**gamma.
#     # Values in the range gamma=(0.5, 2.0) seem to be sensible.
#     iaa.GammaContrast((0.5, 1.5))
# ])

# Make our sequence deterministic.
# We can now apply it to the image and then to the BBs and it will
# lead to the same augmentations.
# IMPORTANT: Call this once PER BATCH, otherwise you will always get the exactly same augmentations for every batch!
seq_det = seq.to_deterministic()

for img in imageList:
    #print(img)
    # Grayscale images must have shape (height, width, 1) each.
    currentimage = skimage.io.imread(dataPath +img).astype(np.uint8)
    # gray2rgb() simply duplicates the gray values over the three color channels.
    currentimage = color.gray2rgb(currentimage)
    image_aug = seq_det.augment_images([currentimage])[0]
    augImgFolder = current_operation + "Images_" + current_class
    createFolder(augImgFolder)
    # Save aug images
    skimage.io.imsave(augImgFolder + '/' + current_class + '.' + str(startID) + '.jpg',image_aug)
    startID += 1
print('Done ' + current_operation)

##################################################
# 5. Fliplr
##################################################

current_operation = "Fliplr"

# Flip/mirror input images horizontally.
   
from imgaug import augmenters as iaa
ia.seed(1)
seq = iaa.Sequential([
        iaa.Fliplr(1.0)
])

# seq = iaa.Sequential([ 
#     # Adjust contrast by scaling each pixel value to (I_ij/255.0)**gamma.
#     # Values in the range gamma=(0.5, 2.0) seem to be sensible.
#     iaa.GammaContrast((0.5, 1.5))
# ])

# Make our sequence deterministic.
# We can now apply it to the image and then to the BBs and it will
# lead to the same augmentations.
# IMPORTANT: Call this once PER BATCH, otherwise you will always get the exactly same augmentations for every batch!
seq_det = seq.to_deterministic()

for img in imageList:
    #print(img)
    # Grayscale images must have shape (height, width, 1) each.
    currentimage = skimage.io.imread(dataPath+img).astype(np.uint8)
    # gray2rgb() simply duplicates the gray values over the three color channels.
    currentimage = color.gray2rgb(currentimage)
    image_aug = seq_det.augment_images([currentimage])[0]
    augImgFolder = current_operation + "Images_" + current_class
    createFolder(augImgFolder)
    # Save aug images
    skimage.io.imsave(augImgFolder + '/' + current_class + '.' + str(startID) + '.jpg',image_aug)
    startID += 1
print('Done ' + current_operation)

##################################################
# 6. Flipud
##################################################

current_operation = "Flipud"

# Flip/mirror input images vertically.
   
from imgaug import augmenters as iaa
ia.seed(1)
seq = iaa.Sequential([
        iaa.Flipud(1.0)
])

# seq = iaa.Sequential([ 
#     # Adjust contrast by scaling each pixel value to (I_ij/255.0)**gamma.
#     # Values in the range gamma=(0.5, 2.0) seem to be sensible.
#     iaa.GammaContrast((0.5, 1.5))
# ])

# Make our sequence deterministic.
# We can now apply it to the image and then to the BBs and it will
# lead to the same augmentations.
# IMPORTANT: Call this once PER BATCH, otherwise you will always get the exactly same augmentations for every batch!
seq_det = seq.to_deterministic()

for img in imageList:
    #print(img)
    # Grayscale images must have shape (height, width, 1) each.
    currentimage = skimage.io.imread(dataPath+img).astype(np.uint8)
    # gray2rgb() simply duplicates the gray values over the three color channels.
    currentimage = color.gray2rgb(currentimage)
    image_aug = seq_det.augment_images([currentimage])[0]
    augImgFolder = current_operation + "Images_" + current_class
    createFolder(augImgFolder)
    # Save aug images
    skimage.io.imsave(augImgFolder + '/' + current_class + '.' + str(startID) + '.jpg',image_aug)
    startID += 1
print('Done ' + current_operation)

##################################################
# 7. Rot90or270Degree
##################################################

current_operation = "Rot90or270Degree"

# Rotates all images by 90 or 270 degrees. 
   
from imgaug import augmenters as iaa
ia.seed(1)
seq = iaa.Sequential([
    iaa.Rot90([1, 3])
])

# seq = iaa.Sequential([ 
#     # Adjust contrast by scaling each pixel value to (I_ij/255.0)**gamma.
#     # Values in the range gamma=(0.5, 2.0) seem to be sensible.
#     iaa.GammaContrast((0.5, 1.5))
# ])

# Make our sequence deterministic.
# We can now apply it to the image and then to the BBs and it will
# lead to the same augmentations.
# IMPORTANT: Call this once PER BATCH, otherwise you will always get the exactly same augmentations for every batch!
seq_det = seq.to_deterministic()

for img in imageList:
    #print(img)
    # Grayscale images must have shape (height, width, 1) each.
    currentimage = skimage.io.imread(dataPath+img).astype(np.uint8)
    # gray2rgb() simply duplicates the gray values over the three color channels.
    currentimage = color.gray2rgb(currentimage)
    image_aug = seq_det.augment_images([currentimage])[0]
    augImgFolder = current_operation + "Images_" + current_class
    createFolder(augImgFolder)
    # Save aug images
    skimage.io.imsave(augImgFolder + '/' + current_class + '.'  + str(startID) + '.jpg',image_aug)
    startID += 1
print('Done ' + current_operation)

Done GaussianNoise
Done GaussianBlur
Done Brightness
Done ContrastNormalization
Done Fliplr
Done Flipud
Done Rot90or270Degree


In [9]:
# Print the newest number after data augmentation for old nonbleeding patients
print(startID)

3121


### New Non Bleeding Patients

In [15]:
# 1001's images are in dataPath
dataPath = "./data/DataSet11/fastAITrain/newNegPatients/"
imgTXT = 'newNeg.txt'
current_class = 'nonbleeding'
startID = 3418

##################################################
# 1. Gaussion Noise
##################################################

## Processing  
trainImageTxtFile = dataPath + imgTXT
imageList = getImageList(trainImageTxtFile)
current_operation = "GaussianNoise"

# Add gaussian noise.
# For 50% of all images, we sample the noise once per pixel.
# For the other 50% of all images, we sample the noise per pixel AND
# channel. This can change the color (not only brightness) of the
# pixels.
   
from imgaug import augmenters as iaa
ia.seed(1)
seq = iaa.Sequential([
        iaa.AdditiveGaussianNoise(loc=0, 
                                  scale=(0.0, 0.05*255), 
                                  per_channel=0.5)
])
# Make our sequence deterministic.
# We can now apply it to the image and then to the BBs and it will
# lead to the same augmentations.
# IMPORTANT: Call this once PER BATCH, otherwise you will always get the exactly same augmentations for every batch!
seq_det = seq.to_deterministic()

for img in imageList:
    # print(img)
    # Grayscale images must have shape (height, width, 1) each.
    #print(os.listdir(dataPath+'images/'))
    currentimage = skimage.io.imread(dataPath + img).astype(np.uint8)
    # gray2rgb() simply duplicates the gray values over the three color channels.
    currentimage = color.gray2rgb(currentimage)
    # bbs = bboxSetupInImage(dataPath , img.rstrip('.jpg') + '.txt',currentimage)
    # Augment BBs and images.
    # As we only have one image and list of BBs, we use
    # [image] and [bbs] to turn both into lists (batches) for the# functions and then [0] to reverse that. In a real experiment, your
    # variables would likely already be lists.
    image_aug = seq_det.augment_images([currentimage])[0]
    # bbs_aug = seq_det.augment_bounding_boxes([bbs])[0]
    augImgFolder = current_operation + "Images_" + current_class
    # augTxTFolder = current_operation + "TXT"
    createFolder(augImgFolder)
    # createFolder(augTxTFolder)
    # Save aug images and bboxes
    skimage.io.imsave(augImgFolder + '/' + current_class + '.'  + str(startID) + '.jpg',image_aug)
    startID += 1
print('Done ' + current_operation)

##################################################
# 2. Gaussion Blur
##################################################

current_operation = "GaussianBlur"

# blur images with a sigma of 0 to 3.0  
from imgaug import augmenters as iaa
ia.seed(1)
seq = iaa.Sequential([
  iaa.GaussianBlur(sigma=(0, 3)) 
])

# Make our sequence deterministic.
# We can now apply it to the image and then to the BBs and it will
# lead to the same augmentations.
# IMPORTANT: Call this once PER BATCH, otherwise you will always get the exactly same augmentations for every batch!
seq_det = seq.to_deterministic()

for img in imageList:
    #print(img)
    # Grayscale images must have shape (height, width, 1) each.
    currentimage = skimage.io.imread(dataPath + img).astype(np.uint8)
    # gray2rgb() simply duplicates the gray values over the three color channels.
    currentimage = color.gray2rgb(currentimage)
    image_aug = seq_det.augment_images([currentimage])[0]
    augImgFolder = current_operation + "Images_" + current_class
    createFolder(augImgFolder)
    # Save aug images
    skimage.io.imsave(augImgFolder + '/' + current_class + '.' + str(startID) + '.jpg',image_aug)
    startID += 1
print('Done ' + current_operation)

##################################################
# 3. Brightness
##################################################

current_operation = "Brightness"

# Strengthen or weaken the contrast in each image.   
from imgaug import augmenters as iaa
ia.seed(1)
seq = iaa.Sequential([
    iaa.Multiply((1.2, 1.5))
])

# seq = iaa.Sequential([ 
#     # Adjust contrast by scaling each pixel value to (I_ij/255.0)**gamma.
#     # Values in the range gamma=(0.5, 2.0) seem to be sensible.
#     iaa.GammaContrast((0.5, 1.5))
# ])

# Make our sequence deterministic.
# We can now apply it to the image and then to the BBs and it will
# lead to the same augmentations.
# IMPORTANT: Call this once PER BATCH, otherwise you will always get the exactly same augmentations for every batch!
seq_det = seq.to_deterministic()

for img in imageList:
    #print(img)
    # Grayscale images must have shape (height, width, 1) each.
    currentimage = skimage.io.imread(dataPath+ img).astype(np.uint8)
    # gray2rgb() simply duplicates the gray values over the three color channels.
    currentimage = color.gray2rgb(currentimage)
    image_aug = seq_det.augment_images([currentimage])[0]
    augImgFolder = current_operation + "Images_" + current_class
    createFolder(augImgFolder)
    # Save aug images
    skimage.io.imsave(augImgFolder + '/' + current_class + '.' + str(startID) + '.jpg',image_aug)
    startID += 1
print('Done ' + current_operation)

##################################################
# 4. ContrastNormalization
##################################################

current_operation = "ContrastNormalization"

# Improve or worsen the contrast of images.

from imgaug import augmenters as iaa
ia.seed(1)
seq = iaa.Sequential([
    iaa.ContrastNormalization((0.8, 1.5), per_channel=True)
])

# seq = iaa.Sequential([ 
#     # Adjust contrast by scaling each pixel value to (I_ij/255.0)**gamma.
#     # Values in the range gamma=(0.5, 2.0) seem to be sensible.
#     iaa.GammaContrast((0.5, 1.5))
# ])

# Make our sequence deterministic.
# We can now apply it to the image and then to the BBs and it will
# lead to the same augmentations.
# IMPORTANT: Call this once PER BATCH, otherwise you will always get the exactly same augmentations for every batch!
seq_det = seq.to_deterministic()

for img in imageList:
    #print(img)
    # Grayscale images must have shape (height, width, 1) each.
    currentimage = skimage.io.imread(dataPath +img).astype(np.uint8)
    # gray2rgb() simply duplicates the gray values over the three color channels.
    currentimage = color.gray2rgb(currentimage)
    image_aug = seq_det.augment_images([currentimage])[0]
    augImgFolder = current_operation + "Images_" + current_class
    createFolder(augImgFolder)
    # Save aug images
    skimage.io.imsave(augImgFolder + '/' + current_class + '.' + str(startID) + '.jpg',image_aug)
    startID += 1
print('Done ' + current_operation)

##################################################
# 5. Fliplr
##################################################

current_operation = "Fliplr"

# Flip/mirror input images horizontally.
   
from imgaug import augmenters as iaa
ia.seed(1)
seq = iaa.Sequential([
        iaa.Fliplr(1.0)
])

# seq = iaa.Sequential([ 
#     # Adjust contrast by scaling each pixel value to (I_ij/255.0)**gamma.
#     # Values in the range gamma=(0.5, 2.0) seem to be sensible.
#     iaa.GammaContrast((0.5, 1.5))
# ])

# Make our sequence deterministic.
# We can now apply it to the image and then to the BBs and it will
# lead to the same augmentations.
# IMPORTANT: Call this once PER BATCH, otherwise you will always get the exactly same augmentations for every batch!
seq_det = seq.to_deterministic()

for img in imageList:
    #print(img)
    # Grayscale images must have shape (height, width, 1) each.
    currentimage = skimage.io.imread(dataPath+img).astype(np.uint8)
    # gray2rgb() simply duplicates the gray values over the three color channels.
    currentimage = color.gray2rgb(currentimage)
    image_aug = seq_det.augment_images([currentimage])[0]
    augImgFolder = current_operation + "Images_" + current_class
    createFolder(augImgFolder)
    # Save aug images
    skimage.io.imsave(augImgFolder + '/' + current_class + '.' + str(startID) + '.jpg',image_aug)
    startID += 1
print('Done ' + current_operation)

##################################################
# 6. Flipud
##################################################

current_operation = "Flipud"

# Flip/mirror input images vertically.
   
from imgaug import augmenters as iaa
ia.seed(1)
seq = iaa.Sequential([
        iaa.Flipud(1.0)
])

# seq = iaa.Sequential([ 
#     # Adjust contrast by scaling each pixel value to (I_ij/255.0)**gamma.
#     # Values in the range gamma=(0.5, 2.0) seem to be sensible.
#     iaa.GammaContrast((0.5, 1.5))
# ])

# Make our sequence deterministic.
# We can now apply it to the image and then to the BBs and it will
# lead to the same augmentations.
# IMPORTANT: Call this once PER BATCH, otherwise you will always get the exactly same augmentations for every batch!
seq_det = seq.to_deterministic()

for img in imageList:
    #print(img)
    # Grayscale images must have shape (height, width, 1) each.
    currentimage = skimage.io.imread(dataPath+img).astype(np.uint8)
    # gray2rgb() simply duplicates the gray values over the three color channels.
    currentimage = color.gray2rgb(currentimage)
    image_aug = seq_det.augment_images([currentimage])[0]
    augImgFolder = current_operation + "Images_" + current_class
    createFolder(augImgFolder)
    # Save aug images
    skimage.io.imsave(augImgFolder + '/' + current_class + '.' + str(startID) + '.jpg',image_aug)
    startID += 1
print('Done ' + current_operation)

##################################################
# 7. Rot90or270Degree
##################################################

current_operation = "Rot90or270Degree"

# Rotates all images by 90 or 270 degrees. 
   
from imgaug import augmenters as iaa
ia.seed(1)
seq = iaa.Sequential([
    iaa.Rot90([1, 3])
])

# seq = iaa.Sequential([ 
#     # Adjust contrast by scaling each pixel value to (I_ij/255.0)**gamma.
#     # Values in the range gamma=(0.5, 2.0) seem to be sensible.
#     iaa.GammaContrast((0.5, 1.5))
# ])

# Make our sequence deterministic.
# We can now apply it to the image and then to the BBs and it will
# lead to the same augmentations.
# IMPORTANT: Call this once PER BATCH, otherwise you will always get the exactly same augmentations for every batch!
seq_det = seq.to_deterministic()

for img in imageList:
    #print(img)
    # Grayscale images must have shape (height, width, 1) each.
    currentimage = skimage.io.imread(dataPath+img).astype(np.uint8)
    # gray2rgb() simply duplicates the gray values over the three color channels.
    currentimage = color.gray2rgb(currentimage)
    image_aug = seq_det.augment_images([currentimage])[0]
    augImgFolder = current_operation + "Images_" + current_class
    createFolder(augImgFolder)
    # Save aug images
    skimage.io.imsave(augImgFolder + '/' + current_class + '.'  + str(startID) + '.jpg',image_aug)
    startID += 1
print('Done ' + current_operation)

Done GaussianNoise
Done GaussianBlur
Done Brightness
Done ContrastNormalization
Done Fliplr
Done Flipud
Done Rot90or270Degree


In [16]:
# New Testing Image For Data Augmentation
print(startID)

4951


In [17]:
# Copy old file to new folder with new image as the folders
for img in imageList:
    #print(img)
    # Grayscale images must have shape (height, width, 1) each.
    currentimage = skimage.io.imread(dataPath+img).astype(np.uint8)
    # gray2rgb() simply duplicates the gray values over the three color channels.
    currentimage = color.gray2rgb(currentimage)
    #image_aug = seq_det.augment_images([currentimage])[0]
    augImgFolder = "Old" + "Images_" + current_class
    createFolder(augImgFolder)
    # Save aug images
    skimage.io.imsave(augImgFolder + '/' + current_class + '.' + str(startID) + '.jpg', currentimage)
    startID += 1

In [18]:
# New Testing Image For Data Augmentation
print(startID)

5170
